<a href="https://colab.research.google.com/github/saurav859/Titanic-Survival-Prediction/blob/main/solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# -*- coding: utf-8 -*-
"""apppp.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/11Kv7eMVsZp8m29GK3vBttAx3RQC_jmOE
"""

# 📦 Import required libraries
import streamlit as st
import pandas as pd
import numpy as np
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# 📍 Model save path
MODEL_PATH = os.path.join(os.getcwd(), 'model.pkl')

# 📌 Streamlit UI
st.title("⚡ Solar Power Prediction App")
uploaded = st.file_uploader("📁 Upload your CSV dataset", type=["csv"])
retrain = st.checkbox("🔄 Retrain model on uploaded data", value=False)

# 📌 Target column selection logic
def select_target_column(df):
    # Prefer common target names
    for name in ['target', 'y', 'label', 'power-generated']:
        if name in df.columns:
            return name
    # Else pick last numeric column
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    return numeric_cols[-1] if numeric_cols else None

# 📌 Main processing pipeline
def process(df, retrain=False, model_path=MODEL_PATH):
    target = select_target_column(df)
    if target is None:
        return {'status': 'error', 'message': '❌ No numeric target column found.'}

    X = df.drop(columns=[target]).select_dtypes(include='number')
    y = df[target]

    if X.shape[1] == 0:
        return {'status': 'error', 'message': '❌ No numeric features available.'}

    # 🔍 Use saved model if available
    if os.path.exists(model_path) and not retrain:
        saved = joblib.load(model_path)
        model = saved['model']
        scaler = saved['scaler']
        X_scaled = scaler.transform(X)
        preds = model.predict(X_scaled)
        return {
            'status': 'ok',
            'action': 'predicted',
            'predictions': preds.tolist(),
            'model_path': model_path
        }

    # 🔧 Train new model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    try:
        model = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
    except:
        model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train_s, y_train)
    preds = model.predict(X_test_s)

    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    joblib.dump({'model': model, 'scaler': scaler, 'target_col': target}, model_path)

    return {
        'status': 'ok',
        'action': 'trained',
        'model_path': model_path,
        'metrics': {'mse': float(mse), 'r2': float(r2)},
        'target_col': target
    }

# 📌 Run pipeline if file is uploaded
if uploaded is not None:
    df = pd.read_csv(uploaded)
    st.write(f"✅ Data loaded: {df.shape[0]} rows × {df.shape[1]} columns")
    st.dataframe(df.head())

    result = process(df, retrain=retrain)

    if result.get('status') == 'ok':
        st.success(f"✅ Action: {result.get('action')}")
        st.write(f"📦 Model saved at: `{result.get('model_path')}`")

        if 'metrics' in result:
            st.subheader("📊 Model Performance")
            st.write(result['metrics'])

        if 'predictions' in result:
            st.subheader("🔮 Predictions (first 20)")
            st.write(result['predictions'][:20])
    else:
        st.error(result.get('message'))
else:
    st.info("📤 Please upload a CSV file to begin.")

2025-11-04 16:26:42.678 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 16:26:42.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 16:26:42.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 16:26:42.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 16:26:42.684 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 16:26:42.685 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 16:26:42.685 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-04 16:26:42.689 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [2]:
%pip install -qq streamlit pandas numpy scikit-learn xgboost joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.9 MB/s eta 0:00:00
